In [90]:
import json
import os

In [116]:
class Contact:
    """
    Classe per rappresentare un contatto nella rubrica telefonica.

    Attributi:
        name (str): Nome del contatto.
        surname (str): Cognome del contatto.
        number (str): Numero di telefono del contatto.
    """
    def __init__(self, name, surname, number):
        self.name = name.title()
        self.surname = surname.title()
        self.number = number

    def __repr__(self):

        """Rappresentazione stringa del contatto per la stampa."""

        return f"{self.name} {self.surname}: {self.number}"

    def __eq__(self, other):

        """
        Confronta due contatti per verificare se sono identici.

        Argomenti:
            other (Contact): L'altro contatto da confrontare.

        Returns:
            boolean: True se i contatti hanno lo stesso nome, cognome e numero.
        """
        if isinstance(other, Contact):
            return (self.name == other.name and
                    self.surname == other.surname and
                    self.number == other.number)
        return False

class Phonebook:
    """
    Classe per la gestione della rubrica telefonica, con funzionalità di
    aggiunta, visualizzazione, ricerca, rimozione e aggiornamento dei contatti.
    """
    def __init__(self):
        self.contacts = []

    def add_contact(self, contact, printit=1):
        """
        Aggiunge un contatto alla rubrica.

        Argomenti:
            contact (Contact): Il contatto da aggiungere.
            printit (int): Flag per stampare o meno il messaggio di successo.
        """
        self.contacts.append(contact)
        if printit==1:
          return print(f"{contact} aggiunto alla rubrica")

    def show_contacts(self):
        """Metodo per visualizzare tutti i contatti nella rubrica."""
        for contact in self.contacts:
            print(contact)

    def search_contact(self, name):
        """
        Cerca un contatto per nome, cognome o nome completo.

        Argomenti:
            name (str): Nome, cognome o nome completo da cercare.

        Returns:
            list: Lista di contatti corrispondenti alla ricerca.
        """
        return [contact for contact in self.contacts if contact.name == name or contact.surname == name or contact.name+' '+contact.surname == name]

    def remove_contact(self, contact, printit=1):
        """
        Rimuove un contatto dalla rubrica.

        Argomenti:
            contact (Contact): Il contatto da rimuovere.
            printit (int): Flag per stampare o meno il messaggio di successo.
        """
        self.contacts.remove(contact)
        if printit==1:
          return print(f"{contact} è stato rimosso dalla rubrica")

    def update_contact(self, contact,newcontact):
        """
        Aggiorna i dettagli di un contatto esistente.

        Args:
            contact (Contact): Il contatto da aggiornare.
            newcontact (Contact): Il contatto con i dettagli aggiornati.
        """
        self.remove_contact(contact, printit=0)
        self.add_contact(newcontact, printit=0)
        return print(f"{contact} è stato aggiornato con {newcontact}")

    def _is_contact(self, contact):
        """
        Verifica se un contatto esiste nella rubrica.

        Args:
            contact (Contact): Il contatto da verificare.

        Returns:
            bool: True se il contatto esiste, False altrimenti.
        """
        if contact in self.contacts:
            return True
        return False

class Files:
    """
    Classe astratta per gestire il salvataggio e caricamento dei file.
    """

    def save(self, data, filename):
        """Metodo astratto per salvare i dati su file."""
        pass

    def load(self, filename):
        """Metodo astratto per caricare i dati da file."""
        pass

class JSONHandler(Files):
    """
    Classe per gestire il salvataggio e caricamento di file JSON.
    """

    def save(self, data, file_path, file_name):
          """
          Salva i contatti in un file JSON.

          Args:
              data (list): Lista di dizionari con i dati dei contatti.
              file_path (str): Percorso della cartella.
              file_name (str): Nome del file JSON.
          """
          full_path = os.path.join(file_path, file_name)
          with open(full_path, 'w') as file:
              json.dump(data, file)
          print(f"File salvato in formato JSON su {full_path}.")

    def load(self, file_path):
          """
          Carica i contatti da un file JSON.

          Args:
              file_path (str): Percorso completo del file JSON.

          Returns:
              list: Lista di dizionari con i dati dei contatti.
          """
          with open(file_path, 'r') as file:
              data = json.load(file)
              return data
          print(f"JSON file caricato da {file_path}.")

# Potenziale classe per gestire i files in csv
class CSVHandler(Files):
    def save(self, data, filename):
        pass

    def load(self, filename):
        pass

class PhonebookManagement:
    """
    Classe per gestire l'interazione dell'utente con la rubrica, tramite
    interfaccia a riga di comando.

    Attributi:
        phonebook (Phonebook): Istanza della rubrica.
        file_handler (Files): Istanza per il salvataggio e caricamento dei file.
    """
    def __init__(self, file_handler: Files):
        self.phonebook = Phonebook()
        self.file_handler = file_handler
        self.unsaved_changes = False
        self.file_open = False

    def input_contact(self):
        """
        Richiede i dettagli del contatto all'utente.

        Returns:
            Contact: Un'istanza di Contact con i dati forniti dall'utente.
        """
        name = input("Nome: ")
        surname = input("Cognome: ")
        number = input("Numero: ")
        try:
          assert(number.isnumeric()), "Numero telefonico non valido"
          return Contact(name, surname, number)
        except AssertionError as e:
          print(e)

    def process(self):
        """
        Gestisce il ciclo di interazione con l'utente tramite menu principale.
        """
        while True:
            print("\nRubrica di Contatti®\n\nMenu:")
            print("1. Aggiungi contatto")
            print("2. Visualizza tutti i contatti")
            print("3. Cerca contatto")
            print("4. Rimuovi contatto")
            print("5. Modifica contatto")
            print("6. Salva rubrica")
            print("7. Carica rubrica")
            print("8. Esci")
            choice = input("Scegli un'opzione: ")

            if choice == "1":
              try:
                print("Inserisci i dati del contatto da aggiungere: ")
                contact = self.input_contact()
                if self.phonebook._is_contact(contact):
                  print("Errore: Il contatto esiste già in rubrica.")
                else:
                  self.phonebook.add_contact(contact)
                  self.unsaved_changes = True
              except AssertionError as e:
                print(e)

            elif choice == "2":
                self.phonebook.show_contacts()
            elif choice == "3":
                name = input("Inserisci nome completo o nome o cognome per la ricerca: ")
                results = self.phonebook.search_contact(name)
                for r in results:
                    print(r)
            elif choice == "4":
                print("Inserisci i dati del contatto da rimuovere: ")
                contact = self.input_contact()
                if self.phonebook._is_contact(contact):
                  remove = input(f"Sei sicuro di voler rimuovere il contatto {contact}? [si/no]")
                  if remove.lower() == 'si':
                    self.phonebook.remove_contact(contact)
                    self.unsaved_changes = True
                  else:
                    continue
                else: print("contatto inesistente")
            elif choice == "5":
                print("Inserisci i dati del contatto da modificare: ")
                contact = self.input_contact()
                if self.phonebook._is_contact(contact):
                  print("Reinserisci i dati con le modifiche: ")
                  new_contact = self.input_contact()
                  self.phonebook.update_contact(contact,new_contact)
                  self.unsaved_changes = True
                else:
                  print("contatto inesistente")
            elif choice == "6":
                try:
                  file_path = input("Inserisci il percorso (ad esempio: C:/percorso/file/): ")
                  file_name = input("Inserisci il nome del file (ad esempio: rubrica.json): ")
                  contacts_data = [{'name': c.name, 'surname': c.surname, 'number': c.number} for c in self.phonebook.contacts]
                  self.file_handler.save(contacts_data, file_path, file_name)
                  self.unsaved_changes = False
                  self.file_open = False
                  clear_phonebook = input("Vuoi pulire la rubrica? [si/no] ")
                  if clear_phonebook.lower() == 'si':
                    self.phonebook = Phonebook()
                except Exception as e:
                  print(e)
            elif choice == "7":
                try:
                  if self.unsaved_changes:
                      print("Errore: Salva le modifiche prima di caricare una nuova rubrica.")
                  elif self.file_open:
                      print("Errore: File aperto, chiudi il file prima di aprirne un altro.")
                  else:
                    path = input("Inserisci il percorso del file JSON (ad esempio: C:/percorso/file/rubrica.json): ")
                    contacts_data = self.file_handler.load(path)
                    self.file_open = True
                    for contact_data in contacts_data:
                        contact = Contact(contact_data['name'], contact_data['surname'], contact_data['number'])
                        self.phonebook.add_contact(contact,printit=0)
                except FileNotFoundError:
                  print("File non trovato.")
                except Exception as e:
                  print(e)
            elif choice == "8":
                if self.unsaved_changes:
                    confirm_exit = input("Ci sono modifiche non salvate. Vuoi uscire comunque? [si/no]: ")
                    if confirm_exit.lower() == 'si':
                        break
                else:
                    break
            else:
                print("Opzione non valida.")


In [ ]:
phonebook_management = PhonebookManagement(JSONHandler())
phonebook_management.process()